# <a id='toc2_'></a>[import libraries](#toc0_)

In [4]:
import pandas as pd
from IPython.display import display
import folium
import numpy as np
import matplotlib.pyplot as plt
from haversine import haversine

In [13]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

In [14]:
path_train = r"/Users/davidchan/Library/CloudStorage/OneDrive-Personal/Important Documents/MASTERS/NUS MSComp (AI) application/CS5228 Knowledge Discovery and Data Mining/Project/train.csv"
path_coe = r"/Users/davidchan/Library/CloudStorage/OneDrive-Personal/Important Documents/MASTERS/NUS MSComp (AI) application/CS5228 Knowledge Discovery and Data Mining/Project/sg-coe-prices.csv"
path_mrt_existing = r"/Users/davidchan/Library/CloudStorage/OneDrive-Personal/Important Documents/MASTERS/NUS MSComp (AI) application/CS5228 Knowledge Discovery and Data Mining/Project/sg-mrt-existing-stations.csv"
path_mrt_planned = r"/Users/davidchan/Library/CloudStorage/OneDrive-Personal/Important Documents/MASTERS/NUS MSComp (AI) application/CS5228 Knowledge Discovery and Data Mining/Project/sg-mrt-planned-stations.csv"
path_primary_schools = r"/Users/davidchan/Library/CloudStorage/OneDrive-Personal/Important Documents/MASTERS/NUS MSComp (AI) application/CS5228 Knowledge Discovery and Data Mining/Project/sg-primary-schools.csv"
path_shopping_malls = r"/Users/davidchan/Library/CloudStorage/OneDrive-Personal/Important Documents/MASTERS/NUS MSComp (AI) application/CS5228 Knowledge Discovery and Data Mining/Project/sg-shopping-malls.csv"
path_stock_prices = r"/Users/davidchan/Library/CloudStorage/OneDrive-Personal/Important Documents/MASTERS/NUS MSComp (AI) application/CS5228 Knowledge Discovery and Data Mining/Project/sg-stock-prices.csv"
path_train_data = r"/Users/davidchan/Library/CloudStorage/OneDrive-Personal/Important Documents/MASTERS/NUS MSComp (AI) application/CS5228 Knowledge Discovery and Data Mining/Project/train.csv"
path_test_data = r"/Users/davidchan/Library/CloudStorage/OneDrive-Personal/Important Documents/MASTERS/NUS MSComp (AI) application/CS5228 Knowledge Discovery and Data Mining/Project/test.csv"
path_list = [path_train, path_coe, path_mrt_existing, path_mrt_planned, path_primary_schools, path_shopping_malls, path_stock_prices, path_train_data, path_test_data]

In [15]:
df_coe_prices = pd.read_csv(path_coe)
df_existing_stations = pd.read_csv(path_mrt_existing)
df_planned_stations = pd.read_csv(path_mrt_planned)
df_primary_schools = pd.read_csv(path_primary_schools)
df_shopping_malls = pd.read_csv(path_shopping_malls)
df_stock_prices = pd.read_csv(path_stock_prices)
df_train = pd.read_csv(path_train_data)
df_test = pd.read_csv(path_test_data)

# <a id='toc3_'></a>[cleaning functions](#toc0_)

In [16]:
def clean_strings(df, columns):
    for col in columns:
        df[col] = df[col].str.lower().str.replace(r'[^a-z0-9]', '', regex=True)
    return df

In [17]:
def identify_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    summary = {
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound,
        'Number of Outliers': outliers.shape[0],
        'Outliers': outliers[column].tolist()
    }

    return summary

In [18]:
def mapping(df, tag):
    center_latitude = 1.3521
    center_longitude = 103.8198
    m = folium.Map(location=[center_latitude, center_longitude], zoom_start=12)  # Adjust zoom_start as needed
    folium.TileLayer('CartoDB Dark_Matter').add_to(m)
    for label, latitude, longitude in zip(df[tag], df['latitude'], df['longitude']):
        tooltip_text = f"Label: {label} Latitude: {latitude}, Longitude: {longitude}"
        folium.Marker([latitude, longitude], icon=folium.Icon(color='red', icon='none'), tooltip=tooltip_text).add_to(m)
    display(m)


In [19]:
def col_dtype_nan_check(df, columns):
    for column in columns:
        print(f"Datatype counts for column '{column}':")
        if column in df.columns:
            datatype_counts = df[column].map(lambda x: type(x).__name__).value_counts()
            nan_count = pd.isna(df[column]).sum()
            print(datatype_counts)
            print(f"Number of NaN values: {nan_count}")
        else:
            print(f"Column '{column}' does not exist in the DataFrame.")
        print()

In [20]:
def one_hot_encode(df, columns):
    return pd.get_dummies(df, columns=columns)

In [21]:
def count_nearby(row, df_comparison, threshold, name_column):
    count = 0
    nearby = []
    for _, location in df_comparison.iterrows():
        location_a = (row['latitude'], row['longitude'])
        location_b = (location['latitude'], location['longitude'])
        distance = haversine(location_a, location_b) # km
        if distance <= threshold:
            count += 1
            nearby.append(location[f"{name_column}"])
    if count_nearby.counter % 5000 == 0:
        print(f"Processed {count_nearby.counter} rows.")
    count_nearby.counter += 1
    return count, ",".join(nearby)

In [126]:
def monthly_avg_close(df):
    df['date'] = pd.to_datetime(df['date'])
    
    df['year_month'] = df['date'].dt.to_period('M')
    
    result = df.groupby(['symbol', 'year_month'])['close'].mean().reset_index()
    
    result.columns = ['symbol', 'year_month', 'avg_close']
    
    return result

In [123]:
def merge_dataframes(rent_df, stock_df):
  
    rent_df['rent_approval_date'] = pd.to_datetime(rent_df['rent_approval_date'])
    stock_df['date'] = pd.to_datetime(stock_df['date'])
    
    rent_df['year_month'] = rent_df['rent_approval_date'].dt.to_period('M')
    stock_df['year_month'] = stock_df['date'].dt.to_period('M')
    
    merged_df = rent_df.merge(stock_df, left_on='year_month', right_on='year_month', how='left')
    
    pivoted_df = merged_df.pivot_table(index='rent_approval_date', columns='symbol', values='close', aggfunc='first').reset_index()
    
    final_df = pd.merge(rent_df, pivoted_df, on='rent_approval_date', how='left')
    
    final_df.drop(columns=['year_month'], inplace=True)
    
    return final_df

# <a id='toc4_'></a>[[sg-mrt-existing-stations] cleaning columns](#toc0_)           [&#8593;](#toc0_)

In [22]:
# create working copy
df_existing_stations_clean = df_existing_stations.copy()
df_existing_stations_clean.head(2)

,code,name,opening_year,latitude,longitude
0,NS1,Jurong East,1990,1.333295,103.742154
1,NS2,Bukit Batok,1990,1.349035,103.749526


In [23]:
# lower, remove spaces and special characters from code and name
df_existing_stations_clean = clean_strings(df_existing_stations_clean, ['code', 'name'])

In [24]:
# check if each code is correctly mapped to name and opening_year

# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

(df_existing_stations_clean['code'] + df_existing_stations_clean['name'] + df_existing_stations_clean['opening_year'].astype(str)).value_counts()

ns1jurongeast1990          1
dt15promenade2013          1
cc27labradorpark2011       1
cc28telokblangah2011       1
cc29harbourfront2011       1
                          ..
ew29jookoon2009            1
ew30gulcircle2017          1
ew31tuascrescent2017       1
ew32tuaswestroad2017       1
te22gardensbythebay2022    1
Length: 162, dtype: int64

In [25]:
# check for outliers in opening_year
identify_outliers(df_existing_stations_clean, 'opening_year')

{'Q1': 1990.0,
 'Q3': 2016.5,
 'IQR': 26.5,
 'Lower Bound': 1950.25,
 'Upper Bound': 2056.25,
 'Number of Outliers': 0,
 'Outliers': []}

In [26]:
# check for outliers in latitude
identify_outliers(df_existing_stations_clean, 'latitude')

{'Q1': 1.29868872290409,
 'Q3': 1.34793886167208,
 'IQR': 0.049250138767990004,
 'Lower Bound': 1.224813514752105,
 'Upper Bound': 1.421814069824065,
 'Number of Outliers': 10,
 'Outliers': [1.4250690629411,
  1.43262603485889,
  1.43605761708128,
  1.44055609916205,
  1.44902668521092,
  1.44307664075699,
  1.4295887655538,
  1.44829245259621,
  1.43605761708128,
  1.42739605281442]}

In [27]:
# check for outliers in longitude
identify_outliers(df_existing_stations_clean, 'longitude')

{'Q1': 103.80675595480925,
 'Q3': 103.86679404069426,
 'IQR': 0.06003808588501158,
 'Lower Bound': 103.71669882598172,
 'Upper Bound': 103.95685116952177,
 'Number of Outliers': 11,
 'Outliers': [103.706064622772,
  103.697418455881,
  103.678085068438,
  103.660530461345,
  103.649078235627,
  103.639616648771,
  103.636991425128,
  103.962374747451,
  103.98788356959,
  103.961472788634,
  103.962374747451]}

# <a id='toc5_'></a>[[sg-mrt-existing-stations] investigating outliers](#toc0_)           [&#8593;](#toc0_)

from above, we see that there may be outliers based on both latitude and longitude.  

latitude: [1.4250690629411,
  1.43262603485889,
  1.43605761708128,
  1.44055609916205,
  1.44902668521092,
  1.44307664075699,
  1.4295887655538,
  1.44829245259621,
  1.43605761708128,
  1.42739605281442]


longitude: [103.706064622772,
  103.697418455881,
  103.678085068438,
  103.660530461345,
  103.649078235627,
  103.639616648771,
  103.636991425128,
  103.962374747451,
  103.98788356959,
  103.961472788634,
  103.962374747451]

In [28]:
# explore outliers for latitude

outliers = [1.4250690629411,
  1.43262603485889,
  1.43605761708128,
  1.44055609916205,
  1.44902668521092,
  1.44307664075699,
  1.4295887655538,
  1.44829245259621,
  1.43605761708128,
  1.42739605281442]

df_outliers = df_existing_stations_clean.loc[df_existing_stations_clean['latitude'].isin(outliers)]
# df_outliers
mapping(df_outliers, 'name')

In [29]:
# explore outliers for longitude

outliers = [103.706064622772,
  103.697418455881,
  103.678085068438,
  103.660530461345,
  103.649078235627,
  103.639616648771,
  103.636991425128,
  103.962374747451,
  103.98788356959,
  103.961472788634,
  103.962374747451]

df_outliers = df_existing_stations_clean.loc[df_existing_stations_clean['longitude'].isin(outliers)]
# df_outliers
mapping(df_outliers, 'name')

In [30]:
# output
df_existing_stations_clean.to_csv('sg-existing-stations-cleaned.csv')

# <a id='toc6_'></a>[[sg-mrt-planned-stations] cleaning columns](#toc0_)         [&#8593;](#toc0_)

In [31]:
# create working copy
df_planned_stations_clean = df_planned_stations.copy()
df_planned_stations_clean.head(2)

,code,name,opening_year,latitude,longitude
0,NaN,Brickland,2030,1.3697,103.7479
1,NaN,Sungei Kadut,2030,1.4133,103.7488


In [32]:
# check dtypes
col_dtype_nan_check(df_planned_stations_clean, df_planned_stations_clean.columns)

Datatype counts for column 'code':
float    74
Name: code, dtype: int64
Number of NaN values: 74

Datatype counts for column 'name':
str    74
Name: name, dtype: int64
Number of NaN values: 0

Datatype counts for column 'opening_year':
str    74
Name: opening_year, dtype: int64
Number of NaN values: 0

Datatype counts for column 'latitude':
float    74
Name: latitude, dtype: int64
Number of NaN values: 0

Datatype counts for column 'longitude':
float    74
Name: longitude, dtype: int64
Number of NaN values: 0



In [33]:
# drop code column
df_planned_stations_clean = df_planned_stations_clean.drop('code',axis = 1)

In [34]:
# lower, remove spaces and special characters from code and name
df_planned_stations_clean = clean_strings(df_planned_stations_clean, ['name'])

In [35]:
# check the values for name and opening year

# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

(df_planned_stations_clean['name'] + df_planned_stations_clean['opening_year'].astype(str)).value_counts()

sungeikadut2030       2
sungeibedok2025       2
tohguan2028           1
nanyanggateway2029    1
loyang2030            1
                     ..
jurongwest2027        1
baharjunction2027     1
boonlay2027           1
gekpoh2027            1
punggol2032           1
Length: 72, dtype: int64

In [36]:
# replace TBA years with the max year
max_year = df_planned_stations_clean[df_planned_stations_clean['opening_year'] != 'TBA']['opening_year'].max()

# Replacing TBA values with the maximum year
df_planned_stations_clean['opening_year'].replace('TBA', max_year, inplace=True)
df_planned_stations_clean['opening_year'].value_counts()

2030    15
2027    11
2040     9
2032     9
2024     8
2028     7
2029     7
2025     5
2026     3
Name: opening_year, dtype: int64

In [37]:
# check for outliers in latitude
identify_outliers(df_planned_stations_clean, 'latitude')

{'Q1': 1.32031,
 'Q3': 1.36591,
 'IQR': 0.04559999999999986,
 'Lower Bound': 1.2519100000000003,
 'Upper Bound': 1.4343099999999998,
 'Number of Outliers': 0,
 'Outliers': []}

In [38]:
# check for outliers in longitude
identify_outliers(df_planned_stations_clean, 'longitude')

{'Q1': 103.74221425,
 'Q3': 103.91277649999999,
 'IQR': 0.17056224999998904,
 'Lower Bound': 103.48637087500002,
 'Upper Bound': 104.16861987499998,
 'Number of Outliers': 0,
 'Outliers': []}

In [39]:
# output
df_planned_stations_clean.to_csv('sg-planned-stations-cleaned.csv')

# <a id='toc7_'></a>[[sg-primary-schools] cleaning columns](#toc0_)        [&#8593;](#toc0_)

In [40]:
# create working copy
df_primary_schools_clean = df_primary_schools.copy()
df_primary_schools_clean.head(2)

,name,latitude,longitude
0,Admiralty Primary School,1.454038,103.817436
1,Ahmad Ibrahim Primary School,1.433153,103.832942


In [41]:
# check dtypes and nan values
col_dtype_nan_check(df_primary_schools_clean, df_primary_schools_clean.columns)

Datatype counts for column 'name':
str    185
Name: name, dtype: int64
Number of NaN values: 0

Datatype counts for column 'latitude':
float    185
Name: latitude, dtype: int64
Number of NaN values: 0

Datatype counts for column 'longitude':
float    185
Name: longitude, dtype: int64
Number of NaN values: 0



In [42]:
# lower, remove spaces and special characters from code and name
df_primary_schools_clean = clean_strings(df_primary_schools_clean, ['name'])

In [43]:
# check the values for name and opening year

# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

(df_primary_schools_clean['name'].astype(str)).value_counts().sort_index()

admiraltyprimaryschool       1
ahmadibrahimprimaryschool    1
aitongschool                 1
alexandraprimaryschool       1
anchorgreenprimaryschool     1
                            ..
yuminprimaryschool           1
yunengprimaryschool          1
zhangdeprimaryschool         1
zhenghuaprimaryschool        1
zhonghuaprimaryschool        1
Name: name, Length: 185, dtype: int64

In [44]:
# check for outliers in latitude
identify_outliers(df_primary_schools_clean, 'latitude')

{'Q1': 1.3347619912564452,
 'Q3': 1.39030227963797,
 'IQR': 0.055540288381524805,
 'Lower Bound': 1.251451558684158,
 'Upper Bound': 1.4736127122102571,
 'Number of Outliers': 0,
 'Outliers': []}

In [45]:
# check for outliers in longitude
identify_outliers(df_primary_schools_clean, 'longitude')

{'Q1': 103.786035065192,
 'Q3': 103.899078139493,
 'IQR': 0.1130430743010038,
 'Lower Bound': 103.61647045374049,
 'Upper Bound': 104.0686427509445,
 'Number of Outliers': 0,
 'Outliers': []}

In [46]:
df_primary_schools_clean.to_csv('sg-primary-schools-cleaned.csv')

# <a id='toc8_'></a>[[sg-shopping-malls] cleaning columns](#toc0_)   [&#8593;](#toc0_)

In [47]:
# create working copy
df_shopping_malls_clean = df_shopping_malls.copy()
df_shopping_malls_clean.head(2)

,name,latitude,longitude
0,100 AM,1.274588,103.843471
1,313@Somerset,1.301385,103.837684


In [48]:
# check dtypes and nan values
col_dtype_nan_check(df_shopping_malls_clean, df_shopping_malls_clean.columns)

Datatype counts for column 'name':
str    163
Name: name, dtype: int64
Number of NaN values: 0

Datatype counts for column 'latitude':
float    163
Name: latitude, dtype: int64
Number of NaN values: 0

Datatype counts for column 'longitude':
float    163
Name: longitude, dtype: int64
Number of NaN values: 0



In [49]:
# lower, remove spaces and special characters from code and name
df_shopping_malls_clean = clean_strings(df_shopping_malls_clean, ['name'])

In [50]:
# check the values for name

# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

(df_shopping_malls_clean['name']).value_counts().sort_index()

100am                  1
313somerset            1
321clementi            1
888plaza               1
admiraltyplace         1
                      ..
woodlandsmart          1
woodlandsnorthplaza    1
yewteepoint            1
yewteesquare           1
zhongshanmall          1
Name: name, Length: 162, dtype: int64

In [51]:
# check for outliers in latitude
identify_outliers(df_shopping_malls_clean, 'latitude')

{'Q1': 1.302990869623295,
 'Q3': 1.3740547896973951,
 'IQR': 0.07106392007410012,
 'Lower Bound': 1.1963949895121448,
 'Upper Bound': 1.4806506698085453,
 'Number of Outliers': 0,
 'Outliers': []}

In [52]:
# check for outliers in longitude
identify_outliers(df_shopping_malls_clean, 'longitude')

{'Q1': 103.79605332855519,
 'Q3': 103.8725576953135,
 'IQR': 0.07650436675831429,
 'Lower Bound': 103.68129677841772,
 'Upper Bound': 103.98731424545097,
 'Number of Outliers': 1,
 'Outliers': [103.989457201894]}

# <a id='toc9_'></a>[[sg-shopping-malls] investigating outliers](#toc0_)   [&#8593;](#toc0_)
from above, we see that there may be an outlier based on longitude.  

longitude: [103.989457201894]

In [53]:
# explore outliers for longitude

outliers = [103.989457201894]

df_outliers = df_shopping_malls_clean.loc[df_shopping_malls_clean['longitude'].isin(outliers)]
# df_outliers
mapping(df_outliers, 'name')

In [54]:
df_shopping_malls_clean.to_csv('sg-shopping-malls-cleaned.csv')

# <a id='toc10_'></a>[[sg-stock-prices] cleaning columns](#toc0_)  [&#8593;](#toc0_)

In [55]:
# create working copy
df_stock_prices_clean = df_stock_prices.copy()
df_stock_prices_clean.head(2)

,name,symbol,date,open,high,low,close,adjusted_close
0,DBS Group,D05.SI,2021-01-04,25.13,25.34,25.01,25.34,22.833
1,DBS Group,D05.SI,2021-01-05,25.22,25.35,25.09,25.35,22.842


In [56]:
# check dtypes and nan values
col_dtype_nan_check(df_stock_prices_clean, df_stock_prices_clean.columns)

Datatype counts for column 'name':
str    35498
Name: name, dtype: int64
Number of NaN values: 0

Datatype counts for column 'symbol':
str    35498
Name: symbol, dtype: int64
Number of NaN values: 0

Datatype counts for column 'date':
str    35498
Name: date, dtype: int64
Number of NaN values: 0

Datatype counts for column 'open':
float    35498
Name: open, dtype: int64
Number of NaN values: 0

Datatype counts for column 'high':
float    35498
Name: high, dtype: int64
Number of NaN values: 0

Datatype counts for column 'low':
float    35498
Name: low, dtype: int64
Number of NaN values: 0

Datatype counts for column 'close':
float    35498
Name: close, dtype: int64
Number of NaN values: 0

Datatype counts for column 'adjusted_close':
float    35498
Name: adjusted_close, dtype: int64
Number of NaN values: 0



In [57]:
# lower, remove spaces and special characters from code and name
df_stock_prices_cleaned = clean_strings(df_stock_prices_clean, ['name', 'symbol'])

In [58]:
# check the values for name and symbol

# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

(df_stock_prices_clean['name'] + df_stock_prices_clean['symbol']).value_counts().sort_index()

ascendasreita17usi                         646
aslanpharmaceuticalsasln                   647
bocaviation2588hk                          632
capitalandascotttrusthmnsi                 646
capitalandinvestmentlimited9cisi           467
capitalandmalltrustc38usi                  646
citydevelopmentsc09si                      646
cmon1792hk                                 632
comfortdelgroc52si                         646
cytomedtherapeuticsgdtc                     73
dbsgroupd05si                              646
flexflex                                   647
fraserslogisticsindustrialtrustbuousi      646
geniusgroupgns                             325
gentingsingaporeg13si                      646
goldenagriresourcese5hsi                   646
grabholdingsgrab                           647
greateasterng07si                          646
grindrodshippinggrin                       647
guardforceaigfai                           460
hphtrusthutchisonportns8usi                646
igginc0799hk 

In [59]:
df_stock_prices_clean.to_csv('sg-stock-prices-cleaned.csv')

# <a id='toc11_'></a>[[sg-coe-prices] cleaning columns](#toc0_)  [&#8593;](#toc0_)

In [60]:
# create working copy
df_coe_prices_clean = df_coe_prices.copy()
df_coe_prices_clean.head(2)

,year,category,month,bidding,price,quota,bids
0,2023,a,july,2,95202,581,728
1,2023,a,july,1,97000,588,756


In [61]:
# check dtypes and nan values
col_dtype_nan_check(df_coe_prices_clean, df_coe_prices_clean.columns)

Datatype counts for column 'year':
int    248
Name: year, dtype: int64
Number of NaN values: 0

Datatype counts for column 'category':
str    248
Name: category, dtype: int64
Number of NaN values: 0

Datatype counts for column 'month':
str    248
Name: month, dtype: int64
Number of NaN values: 0

Datatype counts for column 'bidding':
int    248
Name: bidding, dtype: int64
Number of NaN values: 0

Datatype counts for column 'price':
int    248
Name: price, dtype: int64
Number of NaN values: 0

Datatype counts for column 'quota':
int    248
Name: quota, dtype: int64
Number of NaN values: 0

Datatype counts for column 'bids':
int    248
Name: bids, dtype: int64
Number of NaN values: 0



In [62]:
# lower, remove spaces and special characters from code and name
df_coe_prices_clean = clean_strings(df_coe_prices_clean, ['category', 'month'])

In [63]:
# check the values for category and month

# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

(df_coe_prices_clean['category'] + df_coe_prices_clean['month']).value_counts().sort_index()

aapril        6
aaugust       4
adecember     4
afebruary     6
ajanuary      6
ajuly         6
ajune         6
amarch        6
amay          6
anovember     4
aoctober      4
aseptember    4
bapril        6
baugust       4
bdecember     4
bfebruary     6
bjanuary      6
bjuly         6
bjune         6
bmarch        6
bmay          6
bnovember     4
boctober      4
bseptember    4
capril        6
caugust       4
cdecember     4
cfebruary     6
cjanuary      6
cjuly         6
cjune         6
cmarch        6
cmay          6
cnovember     4
coctober      4
cseptember    4
eapril        6
eaugust       4
edecember     4
efebruary     6
ejanuary      6
ejuly         6
ejune         6
emarch        6
emay          6
enovember     4
eoctober      4
eseptember    4
dtype: int64

In [64]:
df_coe_prices_clean.head(3)

,year,category,month,bidding,price,quota,bids
0,2023,a,july,2,95202,581,728
1,2023,a,july,1,97000,588,756
2,2023,a,june,2,96206,586,751


In [65]:
# check for outliers in latitude
identify_outliers(df_coe_prices_clean, 'year')

{'Q1': 2021.0,
 'Q3': 2022.0,
 'IQR': 1.0,
 'Lower Bound': 2019.5,
 'Upper Bound': 2023.5,
 'Number of Outliers': 0,
 'Outliers': []}

In [66]:
# check for outliers in latitude
identify_outliers(df_coe_prices_clean, 'bidding')

{'Q1': 1.0,
 'Q3': 2.0,
 'IQR': 1.0,
 'Lower Bound': -0.5,
 'Upper Bound': 3.5,
 'Number of Outliers': 0,
 'Outliers': []}

In [67]:
# check for outliers in latitude
identify_outliers(df_coe_prices_clean, 'price')

{'Q1': 51801.0,
 'Q3': 98098.0,
 'IQR': 46297.0,
 'Lower Bound': -17644.5,
 'Upper Bound': 167543.5,
 'Number of Outliers': 0,
 'Outliers': []}

In [68]:
# check for outliers in latitude
identify_outliers(df_coe_prices_clean, 'quota')

{'Q1': 146.0,
 'Q3': 555.25,
 'IQR': 409.25,
 'Lower Bound': -467.875,
 'Upper Bound': 1169.125,
 'Number of Outliers': 0,
 'Outliers': []}

In [69]:
# check for outliers in latitude
identify_outliers(df_coe_prices_clean, 'bids')

{'Q1': 259.25,
 'Q3': 819.75,
 'IQR': 560.5,
 'Lower Bound': -581.5,
 'Upper Bound': 1660.5,
 'Number of Outliers': 0,
 'Outliers': []}

In [70]:
df_coe_prices_clean.to_csv('sg-coe-prices-cleaned.csv')

# [test-data] pre-procesing

In [ ]:
train_data = pd.read_csv('core-data/train.csv')

In [ ]:
# Create a "remaining_year" column for flats
train_data[['rent_approved_year','rent_approved_month']] = train_data['rent_approval_date'].str.split('-', expand=True)
remaining_year = 99 - (train_data['rent_approved_year'].astype(int) - train_data['lease_commence_date'])
train_data['remaining_year'] = remaining_year
train_data['rent_approved_month'] = train_data['rent_approved_month'].astype(int)

In [ ]:
# Drop "furnished" "elevation" "town" "street name" column
train_data = train_data.drop("furnished", axis=1)
train_data = train_data.drop("elevation", axis=1)
train_data = train_data.drop("town", axis=1)
train_data = train_data.drop("street_name", axis=1)
train_data = train_data.drop("rent_approval_date", axis=1)

# Create a "monthly_price_per_sqm" column 
train_data['monthly_price_per_sqm'] = train_data['monthly_rent'] / train_data['floor_area_sqm']
# train_data['year'] = train_data['rent_approval_date'].str.split('-').str[0]

In [ ]:
train_data['flat_type'] = train_data['flat_type'].str.replace(' ', '-')
train_data['flat_type'].unique()

In [ ]:
# Numerical encoding (flat-type, flat-model, planning_area, region)
train_data['flat_type'] = pd.factorize(train_data['flat_type'])[0]
train_data['flat_model'] = pd.factorize(train_data['flat_model'])[0]
train_data['subzone'] = pd.factorize(train_data['subzone'])[0]
train_data['planning_area'] = pd.factorize(train_data['planning_area'])[0]
train_data['region'] = pd.factorize(train_data['region'])[0]

In [ ]:
train_data['yearly_avg_rent'] = train_data.groupby('rent_approved_year')['monthly_rent'].transform('mean')
train_data['monthly_avg_rent_by_region'] = train_data.groupby('region')['monthly_rent'].transform('mean')
train_data['monthly_avg_rent_by_subzone']= train_data.groupby('subzone')['monthly_rent'].transform('mean')
# train_data['monthly_avg_rent_by_town'] = train_data.groupby('town')['monthly_rent'].transform('mean')
train_data['monthly_avg_rent_by_planning_area'] = train_data.groupby('planning_area')['monthly_rent'].transform('mean')
train_data['monthly_avg_rent_by_flat_model'] = train_data.groupby('flat_model')['monthly_rent'].transform('mean')

In [ ]:
train_data['monthly_avg_rent_per_sqm_by_region'] = train_data.groupby('region')['monthly_price_per_sqm'].transform('mean')
train_data['monthly_avg_rent_per_sqm_by_subzone'] = train_data.groupby('subzone')['monthly_price_per_sqm'].transform('mean')
# train_data['monthly_avg_rent_per_sqm_by_town'] = train_data.groupby('town')['monthly_price_per_sqm'].transform('mean')
train_data['monthly_avg_rent_per_sqm_by_planning_area'] = train_data.groupby('planning_area')['monthly_price_per_sqm'].transform('mean')
train_data['monthly_avg_rent_per_sqm_by_flat_model'] = train_data.groupby('flat_model')['monthly_price_per_sqm'].transform('mean')

In [ ]:
# output (DATA CLEANING PART FINISHED)
train_data.to_csv('train-data-cleaned.csv')

# <a id='toc12_'></a>[[train-data-cleaned] adding new variables](#toc0_)

In [74]:
df_train_data = pd.read_csv('train-data-cleaned.csv')
df_train_data_v2 = df_train_data.copy()

In [75]:
threshold = 0.1

count_nearby.counter = 0
print('calculating for existing stations:')
df_train_data_v2[f'n_existing_stations_{threshold}_km'], df_train_data_v2[f'list_existing_stations_{threshold}_km'] = zip(*df_train_data_v2.apply(count_nearby, args = (df_existing_stations_clean, threshold, 'code'), axis = 1))

print('calculating for primary schools:')
df_train_data_v2[f'n_primary_schools_{threshold}_km'], df_train_data_v2[f'list_primary_schools_{threshold}_km'] = zip(*df_train_data_v2.apply(count_nearby, args = (df_primary_schools_clean, threshold, 'name'), axis = 1))

print('calculating for shopping malls:')
df_train_data_v2[f'n_shopping_malls_{threshold}_km'], df_train_data_v2[f'list_shopping_malls_{threshold}_km'] = zip(*df_train_data_v2.apply(count_nearby, args = (df_shopping_malls_clean, threshold, 'name'), axis = 1))

calculating for existing stations:
Processed 0 rows.
Processed 5000 rows.
Processed 10000 rows.
Processed 15000 rows.
Processed 20000 rows.
Processed 25000 rows.
Processed 30000 rows.
Processed 35000 rows.
Processed 40000 rows.
Processed 45000 rows.
Processed 50000 rows.
Processed 55000 rows.
calculating for primary schools:
Processed 60000 rows.
Processed 65000 rows.
Processed 70000 rows.
Processed 75000 rows.
Processed 80000 rows.
Processed 85000 rows.
Processed 90000 rows.
Processed 95000 rows.
Processed 100000 rows.
Processed 105000 rows.
Processed 110000 rows.
Processed 115000 rows.
calculating for shopping malls:
Processed 120000 rows.
Processed 125000 rows.
Processed 130000 rows.
Processed 135000 rows.
Processed 140000 rows.
Processed 145000 rows.
Processed 150000 rows.
Processed 155000 rows.
Processed 160000 rows.
Processed 165000 rows.
Processed 170000 rows.
Processed 175000 rows.


In [76]:
threshold = 0.2

count_nearby.counter = 0
print('calculating for existing stations:')
df_train_data_v2[f'n_existing_stations_{threshold}_km'], df_train_data_v2[f'list_existing_stations_{threshold}_km'] = zip(*df_train_data_v2.apply(count_nearby, args = (df_existing_stations_clean, threshold, 'code'), axis = 1))

print('calculating for primary schools:')
df_train_data_v2[f'n_primary_schools_{threshold}_km'], df_train_data_v2[f'list_primary_schools_{threshold}_km'] = zip(*df_train_data_v2.apply(count_nearby, args = (df_primary_schools_clean, threshold, 'name'), axis = 1))

print('calculating for shopping malls:')
df_train_data_v2[f'n_shopping_malls_{threshold}_km'], df_train_data_v2[f'list_shopping_malls_{threshold}_km'] = zip(*df_train_data_v2.apply(count_nearby, args = (df_shopping_malls_clean, threshold, 'name'), axis = 1))

calculating for existing stations:
Processed 0 rows.
Processed 5000 rows.
Processed 10000 rows.
Processed 15000 rows.
Processed 20000 rows.
Processed 25000 rows.
Processed 30000 rows.
Processed 35000 rows.
Processed 40000 rows.
Processed 45000 rows.
Processed 50000 rows.
Processed 55000 rows.
calculating for primary schools:
Processed 60000 rows.
Processed 65000 rows.
Processed 70000 rows.
Processed 75000 rows.
Processed 80000 rows.
Processed 85000 rows.
Processed 90000 rows.
Processed 95000 rows.
Processed 100000 rows.
Processed 105000 rows.
Processed 110000 rows.
Processed 115000 rows.
calculating for shopping malls:
Processed 120000 rows.
Processed 125000 rows.
Processed 130000 rows.
Processed 135000 rows.
Processed 140000 rows.
Processed 145000 rows.
Processed 150000 rows.
Processed 155000 rows.
Processed 160000 rows.
Processed 165000 rows.
Processed 170000 rows.
Processed 175000 rows.


In [77]:
threshold = 0.5

count_nearby.counter = 0
print('calculating for existing stations:')
df_train_data_v2[f'n_existing_stations_{threshold}_km'], df_train_data_v2[f'list_existing_stations_{threshold}_km'] = zip(*df_train_data_v2.apply(count_nearby, args = (df_existing_stations_clean, threshold, 'code'), axis = 1))

print('calculating for primary schools:')
df_train_data_v2[f'n_primary_schools_{threshold}_km'], df_train_data_v2[f'list_primary_schools_{threshold}_km'] = zip(*df_train_data_v2.apply(count_nearby, args = (df_primary_schools_clean, threshold, 'name'), axis = 1))

print('calculating for shopping malls:')
df_train_data_v2[f'n_shopping_malls_{threshold}_km'], df_train_data_v2[f'list_shopping_malls_{threshold}_km'] = zip(*df_train_data_v2.apply(count_nearby, args = (df_shopping_malls_clean, threshold, 'name'), axis = 1))

calculating for existing stations:
Processed 0 rows.
Processed 5000 rows.
Processed 10000 rows.
Processed 15000 rows.
Processed 20000 rows.
Processed 25000 rows.
Processed 30000 rows.
Processed 35000 rows.
Processed 40000 rows.
Processed 45000 rows.
Processed 50000 rows.
Processed 55000 rows.
calculating for primary schools:
Processed 60000 rows.
Processed 65000 rows.
Processed 70000 rows.
Processed 75000 rows.
Processed 80000 rows.
Processed 85000 rows.
Processed 90000 rows.
Processed 95000 rows.
Processed 100000 rows.
Processed 105000 rows.
Processed 110000 rows.
Processed 115000 rows.
calculating for shopping malls:
Processed 120000 rows.
Processed 125000 rows.
Processed 130000 rows.
Processed 135000 rows.
Processed 140000 rows.
Processed 145000 rows.
Processed 150000 rows.
Processed 155000 rows.
Processed 160000 rows.
Processed 165000 rows.
Processed 170000 rows.
Processed 175000 rows.


In [161]:
df_train_data_v3 = df_train_data_v2.copy()
df_stock_prices_clean_v2 = df_stock_prices_clean.copy()

In [164]:
avg_close_df = monthly_avg_close(df_stock_prices_clean_v2)

# avg_close_df.loc[avg_close_df['symbol']
to_drop = [
    'karo'
    , 'gfai'
    , '9cisi'
    , 'tdcx'
    , 'sopa'
    , 'pgru'
    , 'gns'
    , 'omh'
    , 'gdtc'
]

avg_close_df = avg_close_df.loc[~avg_close_df['symbol'].isin(to_drop)]

avg_close_df = avg_close_df.pivot(index='year_month', columns='symbol', values='avg_close').reset_index()
avg_close_df['year_month'] = avg_close_df['year_month'].astype(str)

avg_close_df.head()
# avg_close_df['symbol'].value_counts()

symbol,year_month,0799hk,1792hk,2588hk,a17usi,ap4si,asln,bn4si,buousi,c07si,...,trirf,u06si,u11si,u14si,u96si,v03si,vc2si,wve,z25si,z74si
0,2020-12,NaN,NaN,NaN,NaN,NaN,9.150000,NaN,NaN,NaN,...,11.030000,NaN,NaN,NaN,NaN,NaN,NaN,7.870000,NaN,NaN
1,2021-01,8.867500,0.074000,64.620000,3.058000,1.428500,10.401316,5.569500,1.473000,21.895500,...,9.676053,2.380000,23.531500,7.668000,1.773000,20.027500,1.608000,9.478947,1.134000,2.429000
2,2021-02,12.934444,0.074278,67.127778,3.057895,1.408421,17.276316,5.063158,1.422632,21.627895,...,7.619474,2.310000,23.841053,7.408421,1.672632,20.006842,1.606316,10.474211,1.122105,2.367895
3,2021-03,10.849565,0.067739,75.969565,2.980870,1.304783,19.139130,5.204783,1.404348,22.496957,...,7.300435,2.447826,25.516087,7.612174,1.802174,19.616957,1.637826,8.999130,1.178261,2.378261
4,2021-04,11.912632,0.063632,72.063158,3.095714,1.380952,16.102381,5.457619,1.468571,23.246190,...,7.618095,2.783810,26.179524,7.870000,2.043333,20.387143,1.741905,6.199048,1.294762,2.491905


In [165]:
df_train_data_v3 = pd.merge(df_train_data_v3, avg_close_df, how='left', left_on='rent_approval_date', right_on='year_month')

df_train_data_v3.drop(columns=['year_month_y'], inplace=True, errors='ignore')

,Unnamed: 0,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,...,trirf,u06si,u11si,u14si,u96si,v03si,vc2si,wve,z25si,z74si
0,0,2021-09,jurong east,257,jurong east street 24,3-room,new generation,67.0,1983,1.344518,...,5.368095,2.650000,25.620000,7.002727,1.875455,18.493636,1.642273,5.825238,1.148182,2.420909
1,1,2022-05,bedok,119,bedok north road,4-room,new generation,92.0,1978,1.330186,...,1.240476,2.516316,29.096316,7.200526,2.842105,17.504211,1.521579,1.458095,1.132632,2.719474
2,2,2022-10,toa payoh,157,lorong 1 toa payoh,3-room,improved,67.0,1971,1.332242,...,0.941429,2.236500,26.367500,6.240500,2.935000,16.229500,1.277500,4.453333,0.922500,2.486000
3,3,2021-08,pasir ris,250,pasir ris street 21,executive,apartment,149.0,1993,1.370239,...,5.265909,2.694286,26.183810,7.176667,1.984286,19.240476,1.486667,5.914091,1.157143,2.346190
4,4,2022-11,kallang/whampoa,34,whampoa west,3-room,improved,68.0,1972,1.320502,...,0.842381,2.236818,29.591364,6.494091,3.076818,17.185000,1.386818,4.129524,0.940682,2.659091


In [168]:
df_train_final = df_train_data_v3 = df_train_data_v3.drop(columns=["Unnamed: 0"])
df_train_final.to_csv('train-data-cleaned-v2.csv')

# [test-data] preprocessing

In [ ]:
test_data = pd.read_csv('core-data/test.csv')

In [ ]:
# Unified duplicated patterns of data
test_data['street_name'] = test_data['street_name'].str.lower()

In [ ]:
#test_data
test_data['street_name'] = test_data['street_name'].str.lower()
duplicates = test_data[test_data.duplicated(keep=False)]
test_data.drop_duplicates(inplace=True)

In [ ]:
#test_data
test_data[['rent_approved_year','rent_approved_month']] = test_data['rent_approval_date'].str.split('-', expand=True)
remaining_year = 99 - (test_data['rent_approved_year'].astype(int) - test_data['lease_commence_date'])
test_data['remaining_year'] = remaining_year

test_data['rent_approved_month'] = test_data['rent_approved_month'].astype(int)

In [ ]:
test_data['flat_type'] = test_data['flat_type'].str.replace(' ', '-')

In [ ]:
# Drop "furnished" "elevation" column
test_data = test_data.drop("furnished", axis=1)
test_data = test_data.drop("elevation", axis=1)
test_data = test_data.drop("town", axis=1)
test_data = test_data.drop("street_name", axis=1)
test_data = test_data.drop("rent_approval_date", axis=1)

In [ ]:
# Numerical encoding (flat-type, flat-model, planning_area, region)
test_data['flat_type'] = pd.factorize(test_data['flat_type'])[0]
test_data['flat_model'] = pd.factorize(test_data['flat_model'])[0]
test_data['subzone'] = pd.factorize(test_data['subzone'])[0]
test_data['planning_area'] = pd.factorize(test_data['planning_area'])[0]
test_data['region'] = pd.factorize(test_data['region'])[0]

In [2]:
test_data.to_csv('test-data-cleaned.csv')


KeyboardInterrupt



# [train-data-cleaned] adding new variables

In [3]:
df_test_data = pd.read_csv('test-data-cleaned.csv')
df_test_data_v2 = df_test_data.copy()

In [ ]:
threshold = 0.1

count_nearby.counter = 0
print('calculating for existing stations:')
df_test_data_v2[f'n_existing_stations_{threshold}_km'], df_test_data_v2[f'list_existing_stations_{threshold}_km'] = zip(*df_test_data_v2.apply(count_nearby, args = (df_existing_stations_clean, threshold, 'code'), axis = 1))

print('calculating for primary schools:')
df_test_data_v2[f'n_primary_schools_{threshold}_km'], df_test_data_v2[f'list_primary_schools_{threshold}_km'] = zip(*df_test_data_v2.apply(count_nearby, args = (df_primary_schools_clean, threshold, 'name'), axis = 1))

print('calculating for shopping malls:')
df_test_data_v2[f'n_shopping_malls_{threshold}_km'], df_test_data_v2[f'list_shopping_malls_{threshold}_km'] = zip(*df_test_data_v2.apply(count_nearby, args = (df_shopping_malls_clean, threshold, 'name'), axis = 1))

calculating for existing stations:
Processed 0 rows.
Processed 5000 rows.
Processed 10000 rows.
Processed 15000 rows.
Processed 20000 rows.
Processed 25000 rows.
Processed 30000 rows.
Processed 35000 rows.
Processed 40000 rows.
Processed 45000 rows.
Processed 50000 rows.
Processed 55000 rows.
calculating for primary schools:
Processed 60000 rows.
Processed 65000 rows.
Processed 70000 rows.
Processed 75000 rows.
Processed 80000 rows.
Processed 85000 rows.
Processed 90000 rows.
Processed 95000 rows.
Processed 100000 rows.
Processed 105000 rows.
Processed 110000 rows.
Processed 115000 rows.
calculating for shopping malls:
Processed 120000 rows.
Processed 125000 rows.
Processed 130000 rows.
Processed 135000 rows.
Processed 140000 rows.
Processed 145000 rows.
Processed 150000 rows.
Processed 155000 rows.
Processed 160000 rows.
Processed 165000 rows.
Processed 170000 rows.
Processed 175000 rows.


In [ ]:
threshold = 0.2

count_nearby.counter = 0
print('calculating for existing stations:')
df_test_data_v2[f'n_existing_stations_{threshold}_km'], df_test_data_v2[f'list_existing_stations_{threshold}_km'] = zip(*df_test_data_v2.apply(count_nearby, args = (df_existing_stations_clean, threshold, 'code'), axis = 1))

print('calculating for primary schools:')
df_test_data_v2[f'n_primary_schools_{threshold}_km'], df_test_data_v2[f'list_primary_schools_{threshold}_km'] = zip(*df_test_data_v2.apply(count_nearby, args = (df_primary_schools_clean, threshold, 'name'), axis = 1))

print('calculating for shopping malls:')
df_test_data_v2[f'n_shopping_malls_{threshold}_km'], df_test_data_v2[f'list_shopping_malls_{threshold}_km'] = zip(*df_test_data_v2.apply(count_nearby, args = (df_shopping_malls_clean, threshold, 'name'), axis = 1))

calculating for existing stations:
Processed 0 rows.
Processed 5000 rows.
Processed 10000 rows.
Processed 15000 rows.
Processed 20000 rows.
Processed 25000 rows.
Processed 30000 rows.
Processed 35000 rows.
Processed 40000 rows.
Processed 45000 rows.
Processed 50000 rows.
Processed 55000 rows.
calculating for primary schools:
Processed 60000 rows.
Processed 65000 rows.
Processed 70000 rows.
Processed 75000 rows.
Processed 80000 rows.
Processed 85000 rows.
Processed 90000 rows.
Processed 95000 rows.
Processed 100000 rows.
Processed 105000 rows.
Processed 110000 rows.
Processed 115000 rows.
calculating for shopping malls:
Processed 120000 rows.
Processed 125000 rows.
Processed 130000 rows.
Processed 135000 rows.
Processed 140000 rows.
Processed 145000 rows.
Processed 150000 rows.
Processed 155000 rows.
Processed 160000 rows.
Processed 165000 rows.
Processed 170000 rows.
Processed 175000 rows.


In [ ]:
threshold = 0.5

count_nearby.counter = 0
print('calculating for existing stations:')
df_test_data_v2[f'n_existing_stations_{threshold}_km'], df_test_data_v2[f'list_existing_stations_{threshold}_km'] = zip(*df_test_data_v2.apply(count_nearby, args = (df_existing_stations_clean, threshold, 'code'), axis = 1))

print('calculating for primary schools:')
df_test_data_v2[f'n_primary_schools_{threshold}_km'], df_test_data_v2[f'list_primary_schools_{threshold}_km'] = zip(*df_test_data_v2.apply(count_nearby, args = (df_primary_schools_clean, threshold, 'name'), axis = 1))

print('calculating for shopping malls:')
df_test_data_v2[f'n_shopping_malls_{threshold}_km'], df_test_data_v2[f'list_shopping_malls_{threshold}_km'] = zip(*df_test_data_v2.apply(count_nearby, args = (df_shopping_malls_clean, threshold, 'name'), axis = 1))

calculating for existing stations:
Processed 0 rows.
Processed 5000 rows.
Processed 10000 rows.
Processed 15000 rows.
Processed 20000 rows.
Processed 25000 rows.
Processed 30000 rows.
Processed 35000 rows.
Processed 40000 rows.
Processed 45000 rows.
Processed 50000 rows.
Processed 55000 rows.
calculating for primary schools:
Processed 60000 rows.
Processed 65000 rows.
Processed 70000 rows.
Processed 75000 rows.
Processed 80000 rows.
Processed 85000 rows.
Processed 90000 rows.
Processed 95000 rows.
Processed 100000 rows.
Processed 105000 rows.
Processed 110000 rows.
Processed 115000 rows.
calculating for shopping malls:
Processed 120000 rows.
Processed 125000 rows.
Processed 130000 rows.
Processed 135000 rows.
Processed 140000 rows.
Processed 145000 rows.
Processed 150000 rows.
Processed 155000 rows.
Processed 160000 rows.
Processed 165000 rows.
Processed 170000 rows.
Processed 175000 rows.


In [ ]:
df_test_data_v3 = df_test_data_v2.copy()
df_stock_prices_clean_v2 = df_stock_prices_clean.copy()

In [ ]:
avg_close_df = monthly_avg_close(df_stock_prices_clean_v2)

# avg_close_df.loc[avg_close_df['symbol']
to_drop = [
    'karo'
    , 'gfai'
    , '9cisi'
    , 'tdcx'
    , 'sopa'
    , 'pgru'
    , 'gns'
    , 'omh'
    , 'gdtc'
]

avg_close_df = avg_close_df.loc[~avg_close_df['symbol'].isin(to_drop)]

avg_close_df = avg_close_df.pivot(index='year_month', columns='symbol', values='avg_close').reset_index()
avg_close_df['year_month'] = avg_close_df['year_month'].astype(str)

avg_close_df.head()
# avg_close_df['symbol'].value_counts()

symbol,year_month,0799hk,1792hk,2588hk,a17usi,ap4si,asln,bn4si,buousi,c07si,...,trirf,u06si,u11si,u14si,u96si,v03si,vc2si,wve,z25si,z74si
0,2020-12,NaN,NaN,NaN,NaN,NaN,9.150000,NaN,NaN,NaN,...,11.030000,NaN,NaN,NaN,NaN,NaN,NaN,7.870000,NaN,NaN
1,2021-01,8.867500,0.074000,64.620000,3.058000,1.428500,10.401316,5.569500,1.473000,21.895500,...,9.676053,2.380000,23.531500,7.668000,1.773000,20.027500,1.608000,9.478947,1.134000,2.429000
2,2021-02,12.934444,0.074278,67.127778,3.057895,1.408421,17.276316,5.063158,1.422632,21.627895,...,7.619474,2.310000,23.841053,7.408421,1.672632,20.006842,1.606316,10.474211,1.122105,2.367895
3,2021-03,10.849565,0.067739,75.969565,2.980870,1.304783,19.139130,5.204783,1.404348,22.496957,...,7.300435,2.447826,25.516087,7.612174,1.802174,19.616957,1.637826,8.999130,1.178261,2.378261
4,2021-04,11.912632,0.063632,72.063158,3.095714,1.380952,16.102381,5.457619,1.468571,23.246190,...,7.618095,2.783810,26.179524,7.870000,2.043333,20.387143,1.741905,6.199048,1.294762,2.491905


In [ ]:
df_test_data_v3 = pd.merge(df_test_data_v3, avg_close_df, how='left', left_on='rent_approval_date', right_on='year_month')

df_test_data_v3.drop(columns=['year_month_y'], inplace=True, errors='ignore')

,Unnamed: 0,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,...,trirf,u06si,u11si,u14si,u96si,v03si,vc2si,wve,z25si,z74si
0,0,2021-09,jurong east,257,jurong east street 24,3-room,new generation,67.0,1983,1.344518,...,5.368095,2.650000,25.620000,7.002727,1.875455,18.493636,1.642273,5.825238,1.148182,2.420909
1,1,2022-05,bedok,119,bedok north road,4-room,new generation,92.0,1978,1.330186,...,1.240476,2.516316,29.096316,7.200526,2.842105,17.504211,1.521579,1.458095,1.132632,2.719474
2,2,2022-10,toa payoh,157,lorong 1 toa payoh,3-room,improved,67.0,1971,1.332242,...,0.941429,2.236500,26.367500,6.240500,2.935000,16.229500,1.277500,4.453333,0.922500,2.486000
3,3,2021-08,pasir ris,250,pasir ris street 21,executive,apartment,149.0,1993,1.370239,...,5.265909,2.694286,26.183810,7.176667,1.984286,19.240476,1.486667,5.914091,1.157143,2.346190
4,4,2022-11,kallang/whampoa,34,whampoa west,3-room,improved,68.0,1972,1.320502,...,0.842381,2.236818,29.591364,6.494091,3.076818,17.185000,1.386818,4.129524,0.940682,2.659091


In [ ]:
df_test_final = df_test_data_v3 = df_test_data_v3.drop(columns=["Unnamed: 0"])
df_test_final.to_csv('test-data-cleaned-v2.csv')